In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [5]:
spark = SparkSession.builder.appName("pyspark-testing").getOrCreate()

In [6]:
filepath = "./data/books3000.csv"
df = spark.read.options(
    header='True',
    inferSchema='True',
    delimiter=","
).csv(filepath)

In [5]:
type(df)

pyspark.sql.dataframe.DataFrame

In [6]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- small_image_url: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- books_count: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: double (nullable = true)
 |-- original_publication_year: integer (nullable = true)
 |-- original_title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- average_rating: double (nullable = true)



In [22]:
df.show(n=5, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------------------
 id                        | 51                                                                          
 author                    | Cassandra Clare                                                             
 title                     | City of Bones (The Mortal Instruments, #1)                                  
 image_url                 | https://images.gr-assets.com/books/1432730315m/256683.jpg                   
 small_image_url           | https://images.gr-assets.com/books/1432730315s/256683.jpg                   
 price                     | 1461                                                                        
 books_count               | 178                                                                         
 isbn                      | 1416914285                                                                  
 isbn13                    | 9.78142E12       

In [24]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show(truncate=False, vertical=True)

-RECORD 0------------------------
 id                        | 0   
 author                    | 0   
 title                     | 0   
 image_url                 | 0   
 small_image_url           | 0   
 price                     | 0   
 books_count               | 0   
 isbn                      | 0   
 isbn13                    | 0   
 original_publication_year | 0   
 original_title            | 0   
 language_code             | 0   
 average_rating            | 0   



In [26]:
df.select(count("id")).show()

+---------+
|count(id)|
+---------+
|     2999|
+---------+



In [32]:
df.select(when(isnan('price').alias("hello!"), True)).show()

+------------------------------------------------+
|CASE WHEN isnan(price) AS `hello!` THEN true END|
+------------------------------------------------+
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                              

In [7]:
df.write.parquet("./data/books.parquet")

In [ ]:
# average_rating
# 
# high rating

In [8]:
parquetFile = spark.read.parquet("./data/books.parquet")

In [11]:
parquetFile.printSchema()

root
 |-- id: integer (nullable = true)
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- small_image_url: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- books_count: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: double (nullable = true)
 |-- original_publication_year: integer (nullable = true)
 |-- original_title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- average_rating: double (nullable = true)



In [10]:
parquetFile.show(vertical=True)parquetFile.show(vertical=True)

-RECORD 0-----------------------------------------
 id                        | 51                   
 author                    | Cassandra Clare      
 title                     | City of Bones (Th... 
 image_url                 | https://images.gr... 
 small_image_url           | https://images.gr... 
 price                     | 1461                 
 books_count               | 178                  
 isbn                      | 1416914285           
 isbn13                    | 9.78142E12           
 original_publication_year | 2007                 
 original_title            | City of Bones        
 language_code             | eng                  
 average_rating            | 4.12                 
-RECORD 1-----------------------------------------
 id                        | 52                   
 author                    | Stephenie Meyer      
 title                     | Eclipse (Twilight... 
 image_url                 | https://images.gr... 
 small_image_url           | ht

In [25]:
average_rating = df.select(f.avg("average_rating")).collect()[0][0]

In [28]:
df.filter(df["average_rating"] >= average_rating).show(vertical=True)

-RECORD 0-----------------------------------------
 id                        | 51                   
 author                    | Cassandra Clare      
 title                     | City of Bones (Th... 
 image_url                 | https://images.gr... 
 small_image_url           | https://images.gr... 
 price                     | 1461                 
 books_count               | 178                  
 isbn                      | 1416914285           
 isbn13                    | 9.78142E12           
 original_publication_year | 2007                 
 original_title            | City of Bones        
 language_code             | eng                  
 average_rating            | 4.12                 
-RECORD 1-----------------------------------------
 id                        | 54                   
 author                    | Douglas Adams        
 title                     | The Hitchhiker's ... 
 image_url                 | https://images.gr... 
 small_image_url           | ht

In [29]:
df.filter(df["average_rating"] < average_rating).show(vertical=True)

-RECORD 0-----------------------------------------
 id                        | 52                   
 author                    | Stephenie Meyer      
 title                     | Eclipse (Twilight... 
 image_url                 | https://images.gr... 
 small_image_url           | https://images.gr... 
 price                     | 2335                 
 books_count               | 185                  
 isbn                      | 316160202            
 isbn13                    | 9.78032E12           
 original_publication_year | 2007                 
 original_title            | Eclipse              
 language_code             | en-US                
 average_rating            | 3.69                 
-RECORD 1-----------------------------------------
 id                        | 53                   
 author                    | Christopher Paolini  
 title                     | Eragon (The Inher... 
 image_url                 | https://images.gr... 
 small_image_url           | ht

In [37]:
df.withColumn("total_price",
f.col("price") * f.col("books_count")).select("total_price").show()

+-----------+
|total_price|
+-----------+
|     260058|
|     431975|
|     455266|
|     790018|
|    1488865|
|     555954|
|     499125|
|     919908|
|     605160|
|      74664|
|      38780|
|     669858|
|    4446440|
|     266375|
|     324386|
|     282210|
|     475558|
|     344960|
|     527588|
|     762496|
+-----------+
only showing top 20 rows



In [38]:
df.filter(
    (df.price >= 2000) & (df.price <3000)
).show(vertical=True)

-RECORD 0-----------------------------------------
 id                        | 52                   
 author                    | Stephenie Meyer      
 title                     | Eclipse (Twilight... 
 image_url                 | https://images.gr... 
 small_image_url           | https://images.gr... 
 price                     | 2335                 
 books_count               | 185                  
 isbn                      | 316160202            
 isbn13                    | 9.78032E12           
 original_publication_year | 2007                 
 original_title            | Eclipse              
 language_code             | en-US                
 average_rating            | 3.69                 
-RECORD 1-----------------------------------------
 id                        | 53                   
 author                    | Christopher Paolini  
 title                     | Eragon (The Inher... 
 image_url                 | https://images.gr... 
 small_image_url           | ht

In [40]:
df.orderBy("original_publication_year", ascending=True).show(vertical=True)

-RECORD 0--------------------------------------------
 id                        | 220                     
 author                    | Mark Cotta Vaz          
 title                     | Twilight: The Com...    
 image_url                 | https://images.gr...    
 small_image_url           | https://images.gr...    
 price                     | 2367                    
 books_count               | 20                      
 isbn                      | 316043133               
 isbn13                    | 9.78032E12              
 original_publication_year | null                    
 original_title            | Twilight: The Com...    
 language_code             | en-US                   
 average_rating            | 4.23                    
-RECORD 1--------------------------------------------
 id                        | 976                     
 author                    | Robert Kapilow, D...    
 title                     | Dr. Seuss's Green...    
 image_url                 |

In [41]:
df.orderBy("author", ascending=True).show(vertical=True)

-RECORD 0-----------------------------------------
 id                        | 2648                 
 author                    | A.A. Milne, Ernes... 
 title                     | The World of Winn... 
 image_url                 | https://s.gr-asse... 
 small_image_url           | https://s.gr-asse... 
 price                     | 1662                 
 books_count               | 58                   
 isbn                      | 525444475            
 isbn13                    | 9.78053E12           
 original_publication_year | 1926                 
 original_title            | Winnie-the-Pooh &... 
 language_code             | eng                  
 average_rating            | 4.43                 
-RECORD 1-----------------------------------------
 id                        | 2877                 
 author                    | A.A. Milne, Ernes... 
 title                     | The Complete Tale... 
 image_url                 | https://s.gr-asse... 
 small_image_url           | ht